In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, SimpleRNN

In [3]:
# Load data
data = pd.read_csv('btc_1h.csv')
data.drop(columns='timestamp',inplace=True)
data.head()

,open,high,low,close,volume
0,4261.48,4313.62,4261.32,4308.83,47.181009
1,4308.83,4328.69,4291.37,4315.32,23.234916
2,4330.29,4345.45,4309.37,4324.35,7.229691
3,4316.62,4349.99,4287.41,4349.99,4.443249
4,4333.32,4377.85,4333.32,4360.69,0.972807


In [4]:
data['target'] = (np.sign(data['close']-data['open'])+1)/2
data['f1'] = data['high']-data['open']
data['f2'] = data['close']-data['low']
data.head()

,open,high,low,close,volume,target,f1,f2
0,4261.48,4313.62,4261.32,4308.83,47.181009,1.0,52.14,47.51
1,4308.83,4328.69,4291.37,4315.32,23.234916,1.0,19.86,23.95
2,4330.29,4345.45,4309.37,4324.35,7.229691,0.0,15.16,14.98
3,4316.62,4349.99,4287.41,4349.99,4.443249,1.0,33.37,62.58
4,4333.32,4377.85,4333.32,4360.69,0.972807,1.0,44.53,27.37


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61447 entries, 0 to 61446
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    61447 non-null  float64
 1   high    61447 non-null  float64
 2   low     61447 non-null  float64
 3   close   61447 non-null  float64
 4   volume  61447 non-null  float64
 5   target  61447 non-null  float64
 6   f1      61447 non-null  float64
 7   f2      61447 non-null  float64
dtypes: float64(8)
memory usage: 3.8 MB


In [6]:
# feature and target separation
X = data.drop(columns=['open','close','target'])
y = data['target']

In [7]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build LSTM model
model1 = Sequential()
model1.add(LSTM(64, activation='relu',input_shape=(X_train.shape[1],1)))
model1.add(Dropout(0.2))
model1.add(Dense(units=8,activation='relu'))
model1.add(Dense(units=1,activation='tanh'))

# Compile model
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


# Train model
history = model1.fit(X_train, y_train, epochs=100, batch_size=512)

# Evaluate model
loss1, accuracy1 = model1.evaluate(X_test,y_test)
print(f'model_1 loss value is: {loss1} \n')
print(f'model_1 accuracy is: {accuracy1}')

Epoch 1/100
85/85 [==============================] - 4s 17ms/step - loss: 0.8878 - accuracy: 0.4955
Epoch 2/100
85/85 [==============================] - 1s 17ms/step - loss: 0.6920 - accuracy: 0.5200
Epoch 3/100
85/85 [==============================] - 2s 18ms/step - loss: 0.6875 - accuracy: 0.5405
Epoch 4/100
85/85 [==============================] - 2s 21ms/step - loss: 0.6802 - accuracy: 0.5600
Epoch 5/100
85/85 [==============================] - 1s 17ms/step - loss: 0.6713 - accuracy: 0.5773
Epoch 6/100
85/85 [==============================] - 1s 17ms/step - loss: 0.6623 - accuracy: 0.5953
Epoch 7/100
85/85 [==============================] - 1s 17ms/step - loss: 0.6540 - accuracy: 0.6076
Epoch 8/100
85/85 [==============================] - 1s 17ms/step - loss: 0.6445 - accuracy: 0.6259
Epoch 9/100
85/85 [==============================] - 2s 18ms/step - loss: 0.6387 - accuracy: 0.6374
Epoch 10/100
85/85 [==============================] - 1s 17ms/step - loss: 0.6293 - accuracy: 0.6490

85/85 [==============================] - 2s 18ms/step - loss: 0.4095 - accuracy: 0.8109
Epoch 83/100
85/85 [==============================] - 2s 19ms/step - loss: 0.4063 - accuracy: 0.8129
Epoch 84/100
85/85 [==============================] - 1s 18ms/step - loss: 0.4075 - accuracy: 0.8122
Epoch 85/100
85/85 [==============================] - 1s 17ms/step - loss: 0.4053 - accuracy: 0.8123
Epoch 86/100
85/85 [==============================] - 1s 17ms/step - loss: 0.4088 - accuracy: 0.8113
Epoch 87/100
85/85 [==============================] - 1s 17ms/step - loss: 0.4051 - accuracy: 0.8131
Epoch 88/100
85/85 [==============================] - 2s 19ms/step - loss: 0.4049 - accuracy: 0.8143
Epoch 89/100
85/85 [==============================] - 2s 18ms/step - loss: 0.4035 - accuracy: 0.8127
Epoch 90/100
85/85 [==============================] - 2s 18ms/step - loss: 0.4063 - accuracy: 0.8131
Epoch 91/100
85/85 [==============================] - 2s 21ms/step - loss: 0.4061 - accuracy: 0.8133
Epo

In [12]:
# Build SimpleRNN model
model2 = Sequential()
model2.add(SimpleRNN(units=64, activation='relu', input_shape=(X_train.shape[1], 1), return_sequences=True))
model2.add(Dropout(0.2))
model2.add(SimpleRNN(units=32, activation='relu', return_sequences=True))
model2.add(Dropout(0.2))
model2.add(SimpleRNN(units=8, activation='relu'))
model2.add(Dense(units=1,activation='tanh'))

# Compile model
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = model2.fit(X_train, y_train, epochs=100, batch_size=512)

# Evaluate model
loss2, accuracy2 = model2.evaluate(X_test,y_test)
print(f'model_2 loss value is: {loss2} \n')
print(f'model_2 accuracy is: {accuracy2}')

Epoch 1/100
85/85 [==============================] - 5s 16ms/step - loss: 0.8364 - accuracy: 0.5267
Epoch 2/100
85/85 [==============================] - 1s 16ms/step - loss: 0.6566 - accuracy: 0.5988
Epoch 3/100
85/85 [==============================] - 1s 16ms/step - loss: 0.6149 - accuracy: 0.6560
Epoch 4/100
85/85 [==============================] - 1s 16ms/step - loss: 0.5867 - accuracy: 0.6864
Epoch 5/100
85/85 [==============================] - 1s 16ms/step - loss: 0.5656 - accuracy: 0.7089
Epoch 6/100
85/85 [==============================] - 1s 17ms/step - loss: 0.5445 - accuracy: 0.7276
Epoch 7/100
85/85 [==============================] - 1s 17ms/step - loss: 0.5270 - accuracy: 0.7367
Epoch 8/100
85/85 [==============================] - 1s 15ms/step - loss: 0.5162 - accuracy: 0.7466
Epoch 9/100
85/85 [==============================] - 1s 15ms/step - loss: 0.5056 - accuracy: 0.7566
Epoch 10/100
85/85 [==============================] - 1s 16ms/step - loss: 0.4936 - accuracy: 0.7630

85/85 [==============================] - 1s 16ms/step - loss: 0.3856 - accuracy: 0.8287
Epoch 83/100
85/85 [==============================] - 1s 15ms/step - loss: 0.3825 - accuracy: 0.8280
Epoch 84/100
85/85 [==============================] - 1s 16ms/step - loss: 0.3845 - accuracy: 0.8268
Epoch 85/100
85/85 [==============================] - 1s 15ms/step - loss: 0.3869 - accuracy: 0.8274
Epoch 86/100
85/85 [==============================] - 2s 18ms/step - loss: 0.3827 - accuracy: 0.8309
Epoch 87/100
85/85 [==============================] - 1s 16ms/step - loss: 0.3887 - accuracy: 0.8279
Epoch 88/100
85/85 [==============================] - 1s 16ms/step - loss: 0.3823 - accuracy: 0.8275
Epoch 89/100
85/85 [==============================] - 1s 17ms/step - loss: 0.3843 - accuracy: 0.8287
Epoch 90/100
85/85 [==============================] - 1s 15ms/step - loss: 0.3853 - accuracy: 0.8293
Epoch 91/100
85/85 [==============================] - 1s 16ms/step - loss: 0.3861 - accuracy: 0.8293
Epo